In [19]:
# init
from os import path
import numpy as np
from pynput import keyboard


smplRate = 44100
timeline_duration = 2
timeline = np.zeros(timeline_duration * smplRate, dtype = np.float32)


# make tone
t0 = .0
duration = 0.2
t = np.array( range( int(t0*smplRate) , int((t0+duration)*smplRate) ) , dtype="float") / smplRate
t2p = t * 2 * np.pi

fade_duration = 0.002
fade_size = int(smplRate*fade_duration)
fade_in = np.linspace(0, 1, fade_size, dtype=np.float32)
fade_out = np.linspace(1, 0, fade_size, dtype=np.float32)

def make_tone(fundamental, harmonics):
    ft2p = fundamental * t2p
    sound = np.sin(ft2p)
    for harmonic in harmonics:
        harmonic_ratios = harmonic[0]
        harmonic_ratios_level = harmonic[1]
        sound += np.sin(ft2p * harmonic_ratios)/harmonic_ratios_level
    sound[:fade_size] *= fade_in
    sound[-fade_size:] *= fade_out
    return sound/10

In [ ]:
# reset keyboard layout
temp_keyboard_layout = []
def on_press(key):
    if key == keyboard.Key.esc:
        # Stop listener
        return False
    try:
        temp_keyboard_layout.append(key.char)
    except AttributeError:
        print('special key {0} pressed'.format(
            key))
with keyboard.Listener(on_press=on_press) as listener:
    listener.join()

new_file_name = input('enter your new keyboar layout name')
if not new_file_name:
    print("I didn't save the file.")
else:
    file_address = path.join('database/keyboard_layouts/' , new_file_name + '.txt')
    with open(file_address,'w') as file:
        file.writelines(temp_keyboard_layout)
        print(new_file_name + " saved as keyboard layout successfully.")

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


I didn't save the file.


In [21]:
# load saved keyboard layout
new_file_name = input('enter your keyboar layout name')
file_address = path.join('database/keyboard_layouts/' , new_file_name + '.txt')
with open(file_address,'r') as file:
    keyboard_layout = list(file.readlines()[0])
    print(new_file_name + " loaded as keyboard layout successfully.")

complete left to right loaded as keyboard layout successfully.


In [28]:
IS_LOOPED = True
harmonics = [(1.5,1),(2.2,2),(3.5,3)]
def scaling_1(point):
    return 200 * 2**(point/12)
def scaling_2(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

In [29]:
# open audio
import pyaudio

position = 0

def callback(in_data, frame_count, time_info, status):
    # If len(data) is less than requested frame_count, PyAudio automatically
    # assumes the stream is finished, and the stream stops.
    global position
    if timeline.size >= (position + frame_count):
        data = timeline[position:position+frame_count].copy()
        if not IS_LOOPED:
            timeline[position:position+frame_count] = 0
        position += frame_count
    else:
        data = timeline[position:].copy()
        if not IS_LOOPED:
            timeline[position:] = 0
        position += frame_count
        position -= timeline.size
        data = np.append(data, timeline[:position])
        if not IS_LOOPED:
            timeline[:position] = 0
    return (data, pyaudio.paContinue)

paud = pyaudio.PyAudio()
stream = paud.open(
            format=pyaudio.paFloat32,
            channels=1,
            rate=smplRate,
            output=True,
            stream_callback=callback
            )



# listen to keyboard
streams = []
def on_press(key):
    try:
        fundamental = scaling_1(scaling_2(keyboard_layout.index(key.char)))
    except AttributeError:
        # print('special key {0} pressed'.format(
        #     key))
        return
    global position
    tone = make_tone(fundamental,harmonics)
    data_size = tone.size
    if timeline.size >= (position + data_size):
        timeline[position:position+data_size] += tone
    else:
        tone.size - timeline.size
        timeline[position:] += tone[:timeline.size-position]
        timeline[:position+tone.size-timeline.size] += tone[timeline.size-position:]
    
def on_release(key):
    if key == keyboard.Key.esc:
        # Stop listener
        stream.close()
        paud.terminate()
        print("stoped.")
        return False

# Collect events until released
# with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#     listener.join()

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()
print("started...")

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [7]:
# closing audio in case of emergency
stream.close()
paud.terminate()

In [27]:
# clear timline
timeline = np.zeros(timeline_duration * smplRate, dtype = np.float32)

In [6]:
#save timeline
new_file_name = input('enter your new timeline name')
file_address = path.join('database/saved_sounds/' , new_file_name + '.npy')
with open(file_address,'wb') as file:
    np.save(file,timeline)
    print(new_file_name + " saved as timeline successfully.")


saved_sounds/di di di.npy


In [9]:
#load timeline
new_file_name = input('enter your timeline name')
file_address = path.join('database/saved_sounds/' , new_file_name + '.npy')
with open(file_address,'rb') as file:
    timeline += np.load(file)
    print(new_file_name + " loaded as timeline successfully.")